# "Tanzania Water Pump Prediction"

In [1]:
import pandas as pd
import numpy as np 

## 파일불러오기 (Train, Train Label, Test)

In [2]:
train_values = pd.read_csv('watertrainingset.csv',sep=',')

In [3]:
test_values = pd.read_csv('watertestset.csv',sep=',')

In [4]:
train_labels=pd.read_csv('watertraininglabel.csv',sep=',')

In [5]:
print(train_values.shape)
print(test_values.shape)
print(train_labels.shape)

(59400, 40)
(14850, 40)
(59400, 2)


In [6]:
train = pd.merge(train_values, train_labels, on = 'id')

# EDA

### ◆기본통계확인

In [7]:
print(train.status_group.value_counts())

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64


### ◆ 연속형 변수 기본 정보 출력

In [8]:
train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


construnction year, population, amount_tsh outlier 처리해야 됨. 

이 외 변수는 tableau의 시각자료를 활용한다 

## 데이터 전처리 

In [9]:
train_set =train
test_set=test_values#혹시 몰라 복사해서 활용

### ◆ 변수 삭제

<삭제할 변수기준>
1. Redundant Data
2. 필요없을 것으로 자의적 판단
3. 거의 모든 변수와의 상관관계가 0인 연속형 변수

### 1. Redundant Data 체크목록

1. Extraction > type 삭제 
2. Source > type,class삭제 
3. waterpoint > group 삭제 (이미 소분류에 대분류가 언급되어 있어서)
4. management > group 삭제 
5. payment > type삭제 
6. quality > quality group 삭제 
7. quantity > quantity group
8. location 관련 > district, region code, lga, ward, subvillage

In [10]:
train_set=train_set.drop(['extraction_type_group','extraction_type_class','source_type','source_class','waterpoint_type_group','management_group','payment_type','quality_group','quantity_group','district_code','region_code','lga','ward','subvillage'],axis=1)
test_set=test_set.drop(['extraction_type_group','extraction_type_class','source_type','source_class','waterpoint_type_group','management_group','payment_type','quality_group','quantity_group','district_code','region_code','lga','ward','subvillage'],axis=1)

### 2. 필요없는 변수 자의적 판단

1. recored_by :  누가 데이터를 입력했는지 
2. data_recorded: 언제 데이터를 입력했는지
3. wpt_name : id가 있으므로 이름은 필요없음 
4. scheme_name : scheme_management 로 대체 가능 
5. id

In [11]:
train_set = train_set.drop(['id','recorded_by','date_recorded','wpt_name','scheme_name'],axis=1)
test_set = test_set.drop(['id','recorded_by','date_recorded','wpt_name','scheme_name'],axis=1)

### 3. 거의 모든 변수와의 상관관계가 0인 연속형 변수

In [12]:
train.corr()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,1.000000,-0.005321,-0.004692,-0.001348,0.001718,-0.002629,-0.003028,-0.003044,-0.002813,-0.002082
amount_tsh,-0.005321,1.000000,0.076650,0.022134,-0.052670,0.002944,-0.026813,-0.023599,0.016288,0.067915
gps_height,-0.004692,0.076650,1.000000,0.149155,-0.035751,0.007237,-0.183521,-0.171233,0.135003,0.658727
longitude,-0.001348,0.022134,0.149155,1.000000,-0.425802,0.023873,0.034197,0.151398,0.086590,0.396732
latitude,0.001718,-0.052670,-0.035751,-0.425802,1.000000,0.006837,-0.221018,-0.201020,-0.022152,-0.245278
num_private,-0.002629,0.002944,0.007237,0.023873,0.006837,1.000000,-0.020377,-0.004478,0.003818,0.026056
region_code,-0.003028,-0.026813,-0.183521,0.034197,-0.221018,-0.020377,1.000000,0.678602,0.094088,0.031724
district_code,-0.003044,-0.023599,-0.171233,0.151398,-0.201020,-0.004478,0.678602,1.000000,0.061831,0.048315
population,-0.002813,0.016288,0.135003,0.086590,-0.022152,0.003818,0.094088,0.061831,1.000000,0.260910
construction_year,-0.002082,0.067915,0.658727,0.396732,-0.245278,0.026056,0.031724,0.048315,0.260910,1.000000


1. num_private 삭제 : num_private은 거의 0에 수렴하여 모든 변수와의 관계가 없다고 봐도 무방. 

In [13]:
train_set= train_set.drop(['num_private'],axis=1)
test_set= test_set.drop(['num_private'],axis=1)

### ◆ 결측치 제거 및 변환

In [14]:
train_set.isnull().sum()

amount_tsh              0
funder               3635
gps_height              0
installer            3655
longitude               0
latitude                0
basin                   0
region                  0
population              0
public_meeting       3334
scheme_management    3877
permit               3056
construction_year       0
extraction_type         0
management              0
payment                 0
water_quality           0
quantity                0
source                  0
waterpoint_type         0
status_group            0
dtype: int64

train_set=train_set.dropna(subset=['public_meeting', 'permit'], how='any') # true/false 열 제거 
test_set=test_set.dropna(subset=['public_meeting', 'permit'], how='any')

In [15]:
'unknown' in train_set.installer
'unknown' in train_set.funder
'unknown' in train_set.scheme_management
  #결측치를 unknown 으로 변경 

False

In [16]:
train_set = train_set.fillna('unknown')
test_set = test_set.fillna('unknown')

In [17]:
train_set.isnull().sum() #확인

amount_tsh           0
funder               0
gps_height           0
installer            0
longitude            0
latitude             0
basin                0
region               0
population           0
public_meeting       0
scheme_management    0
permit               0
construction_year    0
extraction_type      0
management           0
payment              0
water_quality        0
quantity             0
source               0
waterpoint_type      0
status_group         0
dtype: int64

###  ◆너무 많은 level을 가진 범주형 변수 레벨 고치기

funder, installer

#### 레벨 고치는 함수 

In [18]:
def cut_levels(x, threshold, new_value):
    value_counts=x.value_counts()
    labels = value_counts.index[value_counts<threshold]
    x[np.in1d(x,labels)]=new_value

In [19]:
cut_levels(train_set.funder,100, 'others')
cut_levels(test_set.funder, 100, 'others')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
cut_levels(train_set.installer,100, 'others')
cut_levels(test_set.installer, 100, 'others')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
train_set.head() #확인작업 

,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region,population,public_meeting,...,permit,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,status_group
0,6000.0,Roman,1390,others,34.938093,-9.856322,Lake Nyasa,Iringa,109,True,...,False,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,functional
1,0.0,others,1399,others,34.698766,-2.147466,Lake Victoria,Mara,280,unknown,...,True,2010,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,functional
2,25.0,others,686,World vision,37.460664,-3.821329,Pangani,Manyara,250,True,...,True,2009,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,functional
3,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,58,True,...,True,1986,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,non functional
4,0.0,others,0,Artisan,31.130847,-1.825359,Lake Victoria,Kagera,0,True,...,True,0,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,functional


### ◆ outlier 처리

In [22]:
train_set.describe()

,amount_tsh,gps_height,longitude,latitude,population,construction_year
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000
mean,317.650385,668.297239,34.077427,-5.706033e+00,179.909983,1300.652475
std,2997.574558,693.116350,6.567432,2.946019e+00,471.482176,951.620547
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,0.000000
25%,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,0.000000
50%,0.000000,369.000000,34.908743,-5.021597e+00,25.000000,1986.000000
75%,20.000000,1319.250000,37.178387,-3.326156e+00,215.000000,2004.000000
max,350000.000000,2770.000000,40.345193,-2.000000e-08,30500.000000,2013.000000


#### construction year = 1990 이후에는 functional 한 것이 눈에 띄게 늘어남. 나머지는 늘 비슷비슷

In [23]:
train_set.construction_year.value_counts() #0을 처리해야 함. 1990 이상은 1로 1990 이하는 0으로 범주형 데이터로 변환 

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

In [24]:
train_set.construction_year[train_set.construction_year ==0]=1990
test_set.construction_year[test_set.construction_year==0]=1990

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


0 년을 1990년으로 change

#### amount_tsh, population 은 0 값이 너무 많아 skewed되어 있음. log 값을 취해준다 

In [25]:
train_set['amount_tsh']=train_set.amount_tsh.apply(lambda x:np.log(x+1))
train_set['population']=train_set.population.apply(lambda x:np.log(x+1))

In [26]:
train_set.describe()

,amount_tsh,gps_height,longitude,latitude,population,construction_year
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000
mean,1.585235,668.297239,34.077427,-5.706033e+00,2.819701,1994.438838
std,2.636662,693.116350,6.567432,2.946019e+00,2.662184,10.576678
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1960.000000
25%,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,1990.000000
50%,0.000000,369.000000,34.908743,-5.021597e+00,3.258097,1990.000000
75%,3.044522,1319.250000,37.178387,-3.326156e+00,5.375278,2004.000000
max,12.765691,2770.000000,40.345193,-2.000000e-08,10.325515,2013.000000


In [27]:
test_set['amount_tsh']=test_set.amount_tsh.apply(lambda x:np.log(x+1))
test_set['population']=test_set.population.apply(lambda x:np.log(x+1))

In [28]:
train_set.head()
test_set.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region,population,public_meeting,scheme_management,permit,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type
0,0.000000,others,1996,others,35.290799,-4.059696,Internal,Manyara,5.774552,True,Parastatal,True,2012,other,parastatal,never pay,soft,seasonal,rainwater harvesting,other
1,0.000000,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Pangani,Arusha,5.707110,True,VWC,True,2000,gravity,vwc,never pay,soft,insufficient,spring,communal standpipe
2,0.000000,unknown,1567,unknown,34.767863,-5.004344,Internal,Singida,6.216606,True,VWC,unknown,2010,other,vwc,never pay,soft,insufficient,rainwater harvesting,other
3,0.000000,others,267,others,38.058046,-9.418672,Ruvuma / Southern Coast,Lindi,5.525453,unknown,VWC,True,1987,other,vwc,unknown,soft,dry,shallow well,other
4,6.216606,others,1260,others,35.006123,-10.950412,Ruvuma / Southern Coast,Ruvuma,4.110874,unknown,Water Board,True,2000,gravity,water board,pay monthly,soft,enough,spring,communal standpipe


### 모델을 돌리기 위해서 숫자형으로 바꿔주기전에 복사본 만들기

In [29]:
train_model = train_set
test_model = test_set 

In [30]:
#numeric 변환전 
train_set.to_csv('pump_train_set.csv',index=False)
test_set.to_csv('pump_test_set.csv',index=False)

In [31]:
train_set

,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region,population,public_meeting,...,permit,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,status_group
0,8.699681,Roman,1390,others,34.938093,-9.856322e+00,Lake Nyasa,Iringa,4.700480,True,...,False,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,functional
1,0.000000,others,1399,others,34.698766,-2.147466e+00,Lake Victoria,Mara,5.638355,unknown,...,True,2010,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,functional
2,3.258097,others,686,World vision,37.460664,-3.821329e+00,Pangani,Manyara,5.525453,True,...,True,2009,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,functional
3,0.000000,Unicef,263,UNICEF,38.486161,-1.115530e+01,Ruvuma / Southern Coast,Mtwara,4.077537,True,...,True,1986,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,non functional
4,0.000000,others,0,Artisan,31.130847,-1.825359e+00,Lake Victoria,Kagera,0.000000,True,...,True,1990,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,functional
5,3.044522,Mkinga Distric Coun,0,DWE,39.172796,-4.765587e+00,Pangani,Tanga,0.693147,True,...,True,2009,submersible,vwc,pay per bucket,salty,enough,other,communal standpipe multiple,functional
6,0.000000,Dwsp,0,DWSP,33.362410,-3.766365e+00,Internal,Shinyanga,0.000000,True,...,True,1990,swn 80,vwc,never pay,soft,enough,machine dbh,hand pump,non functional
7,0.000000,Rwssp,0,DWE,32.620617,-4.226198e+00,Lake Tanganyika,Shinyanga,0.000000,True,...,True,1990,nira/tanira,wug,unknown,milky,enough,shallow well,hand pump,non functional
8,0.000000,Wateraid,0,others,32.711100,-5.146712e+00,Lake Tanganyika,Tabora,0.000000,True,...,True,1990,india mark ii,vwc,never pay,salty,seasonal,machine dbh,hand pump,non functional
9,0.000000,others,0,Artisan,30.626991,-1.257051e+00,Lake Victoria,Kagera,0.000000,True,...,True,1990,nira/tanira,vwc,never pay,soft,enough,shallow well,hand pump,functional


### ◆ 모든 문자형 변수 = numeric으로 바꾸기 (그래야 모델링 가능)

In [32]:
train_model.status_group.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

funder, installer,basin,region, public_meeting, , scheme_management, permit, extraction_type, management, payment,water_qualityquantity,           
source ,    waterpoint_type      status_group    

status_group 바꾸기 

In [33]:
def transformation(df, column_name):
    unique_values = set(df[column_name].tolist())
    transform_dict = {}
    for i, value in enumerate(unique_values):
        transform_dict[value]=i
    
    def label_map(x):
        return transform_dict[x]
    df[column_name]=df[column_name].apply(label_map)
    return df 

In [34]:
watercolumns_to_transform = ["funder","installer","basin","region","public_meeting","scheme_management","permit","extraction_type","management","payment","water_quality","quantity","source","waterpoint_type","status_group"]
#test_model 변환위해 status_group 제거 
watercolumns_to_transform2 = ["funder","installer","basin","region","public_meeting","scheme_management","permit","extraction_type","management","payment","water_quality","quantity","source","waterpoint_type"]

In [35]:
for column in watercolumns_to_transform:
    train_model = transformation(train_model,column)

In [36]:
 for column in watercolumns_to_transform2:
    test_model = transformation(test_model,column)

In [37]:
train_model.corr()["status_group"]

amount_tsh           0.155505
funder              -0.018784
gps_height           0.096408
installer            0.002172
longitude            0.071635
latitude            -0.021020
basin                0.043406
region               0.069689
population           0.040964
public_meeting       0.042963
scheme_management   -0.030732
permit               0.021958
construction_year    0.198656
extraction_type      0.077097
management          -0.079755
payment             -0.137575
water_quality        0.056713
quantity            -0.150845
source              -0.012486
waterpoint_type      0.112813
status_group         1.000000
Name: status_group, dtype: float64

In [38]:
train_model.status_group

0        2
1        2
2        2
3        1
4        2
5        2
6        1
7        1
8        1
9        2
10       2
11       2
12       2
13       2
14       2
15       2
16       1
17       1
18       0
19       2
20       2
21       2
22       0
23       2
24       2
25       0
26       2
27       2
28       1
29       2
        ..
59370    2
59371    1
59372    1
59373    2
59374    2
59375    2
59376    1
59377    1
59378    2
59379    2
59380    1
59381    1
59382    1
59383    2
59384    1
59385    2
59386    2
59387    2
59388    2
59389    2
59390    2
59391    1
59392    1
59393    2
59394    1
59395    2
59396    2
59397    2
59398    2
59399    2
Name: status_group, dtype: int64

In [39]:
train_set.status_group

0        2
1        2
2        2
3        1
4        2
5        2
6        1
7        1
8        1
9        2
10       2
11       2
12       2
13       2
14       2
15       2
16       1
17       1
18       0
19       2
20       2
21       2
22       0
23       2
24       2
25       0
26       2
27       2
28       1
29       2
        ..
59370    2
59371    1
59372    1
59373    2
59374    2
59375    2
59376    1
59377    1
59378    2
59379    2
59380    1
59381    1
59382    1
59383    2
59384    1
59385    2
59386    2
59387    2
59388    2
59389    2
59390    2
59391    1
59392    1
59393    2
59394    1
59395    2
59396    2
59397    2
59398    2
59399    2
Name: status_group, dtype: int64

### 전처리된 파일저장

In [40]:

#numeric 변환 후 
train_model.to_csv('pump_train_model_set.csv',index=False)
test_model.to_csv('pump_test_model_set.csv',index=False)

In [41]:
train_model

,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region,population,public_meeting,...,permit,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,status_group
0,8.699681,23,1390,46,34.938093,-9.856322e+00,2,16,4.700480,1,...,0,1999,9,10,3,6,1,5,6,2
1,0.000000,65,1399,46,34.698766,-2.147466e+00,0,7,5.638355,2,...,1,2010,9,6,5,6,4,1,6,2
2,3.258097,65,686,7,37.460664,-3.821329e+00,8,19,5.525453,1,...,1,2009,9,10,2,6,1,6,4,2
3,0.000000,4,263,5,38.486161,-1.115530e+01,6,13,4.077537,1,...,1,1986,5,10,5,6,3,7,4,1
4,0.000000,65,0,12,31.130847,-1.825359e+00,0,4,0.000000,1,...,1,1990,9,4,5,6,2,1,6,2
5,3.044522,32,0,68,39.172796,-4.765587e+00,8,11,0.693147,1,...,1,2009,5,10,2,2,1,2,4,2
6,0.000000,50,0,18,33.362410,-3.766365e+00,7,14,0.000000,1,...,1,1990,11,10,5,6,1,7,3,1
7,0.000000,63,0,68,32.620617,-4.226198e+00,1,14,0.000000,1,...,1,1990,15,6,6,5,1,4,3,1
8,0.000000,55,0,46,32.711100,-5.146712e+00,1,12,0.000000,1,...,1,1990,14,10,5,2,2,7,3,1
9,0.000000,65,0,12,30.626991,-1.257051e+00,0,4,0.000000,1,...,1,1990,15,10,5,6,1,4,3,2
